In [1]:
import numpy as np
from sympy import *
from numpy.linalg import inv,det
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

In [2]:
x0 = np.array([[6.0,0.0,0.0,6.0],[4.0,4.0,0.0,0.0]])

In [3]:
X= np.array([[1.25,0],[0,1.125]])

In [4]:
xt = X.dot(x0)
print(xt)

[[7.5 0.  0.  7.5]
 [4.5 4.5 0.  0. ]]


In [5]:

piola_stress = np.array([[100,0],[0,60]])
print(piola_stress)

[[100   0]
 [  0  60]]


$$  \textbf{C} =
\frac{E}{(1-\nu^2)} \left(\begin{array}{cc} 
1 & \nu & 0\\
\nu & 1 & 0 \\
0 & 0 & \frac{1-\nu}{2}
\end{array}\right)
$$ 

In [6]:
E = symbols('E')
v=0.3
stiffness_m = E/((1-v*v))*np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])
print(stiffness_m)

[[1.0989010989011*E 0.32967032967033*E 0]
 [0.32967032967033*E 1.0989010989011*E 0]
 [0 0 0.384615384615385*E]]


Interpolation functions of form: 
$$ h_1 = \frac{1}{4}(1+r)(1+s)$$
$h_{1,1}$ is the derivative w.r.t. $\frac{\partial}{\partial^0x_1}$ :
$$h_{1,1}=\frac{\partial h_1}{\partial r}\frac{\partial r}{\partial^0x_1} = \frac{1}{12}(1+s), \frac{\partial r}{\partial^0x_1} = \frac{1}{3}$$
resp.:
$$h_{1,2}=\frac{\partial h_1}{\partial s}\frac{\partial s}{\partial^0x_2} = \frac{1}{8}(1+r), \frac{\partial s}{\partial^0x_2} = \frac{1}{2}$$

In [7]:
x, y, h = symbols('x y h')
scale=[3,2];
def h1(x,y): return 1/4*(1+x)*(1+y)
def h2(x,y): return 1/4*(1-x)*(1+y)
def h3(x,y): return 1/4*(1-x)*(1-y)
def h4(x,y): return 1/4*(1+x)*(1-y)
def hprime(h,x,y,j):
    dim =[x,y]
    return diff(h(x,y),dim[j]) / scale[j]
list_of_interp_fcts = np.array([h1,h2,h3,h4])

$$ {}^t_0\textbf{B}_{L0} = \begin{bmatrix}
h_{1,1} & 0 &|& ...\\
0 &h_{1,2} &|& ...\\
h_{1,2} & h_{1,1} &|& ...
\end{bmatrix} $$

In [8]:
BL = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL=np.concatenate((BL,np.array([
        [hprime(hi,x,y,0),0,hprime(hi,x,y,1)],
        [0,hprime(hi,x,y,1),hprime(hi,x,y,0)]
])))
BL =BL.T
print(BL[0][0])

0.0833333333333333*y + 0.0833333333333333


$$ {}^tu^k = {}^tx^k - {}^0x^k $$

In [9]:
u = xt-x0
print(u)

[[1.5 0.  0.  1.5]
 [0.5 0.5 0.  0. ]]


$$l_{ij} = \sum_{k=1}^{N} h_{k,j} {}^tu_i^k$$

In [10]:
def lij(u,n,i,j):
    h=[h1,h2,h3,h4]
    lsum=0
    for k in range(n):
        lsum = lsum + hprime(h[k],x,y,j)*u[i][k]
    return lsum

$$ {}^t_0\textbf{B}_{L1} = \begin{bmatrix}
l_{11}h_{1,1} & l_{21}h_{1,1} &|& ...\\
l_{12}h_{1,2} & l_{22}h_{1,2} &|& ...\\
l_{11}h_{1,2}+l_{12}h_{1,1} & l_{21}h_{1,2}+l_{22}h_{1,1} &|& ...
\end{bmatrix} $$

In [11]:
BL1 = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL1=np.concatenate((BL1,np.array([
        [hprime(hi,x,y,0)*lij(u,4,0,0),hprime(hi,x,y,1)*lij(u,4,0,1),hprime(hi,x,y,1)*lij(u,4,0,0)+hprime(hi,x,y,0)*lij(u,4,0,1)],
        [hprime(hi,x,y,0)*lij(u,4,1,0),hprime(hi,x,y,1)*lij(u,4,1,1),hprime(hi,x,y,1)*lij(u,4,1,0)+hprime(hi,x,y,0)*lij(u,4,1,1)]
])))
BL1 =BL1.T
print(BL1[0][0])

0.0208333333333333*y + 0.0208333333333333


$$ {}^t_0\textbf{B}_L = {}^t_0\textbf{B}_{L0} + {}^t_0\textbf{B}_{L1} $$

In [12]:
BL=BL+BL1
print(round_expr(BL[0][0],4))

0.1042*y + 0.1042


$$ J = \frac{\partial^0x_1}{\partial r}\frac{\partial^0x_2}{\partial s} = 6 $$

$$ {}^t_0\textbf{K}_L = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{L}\textbf{C}{}_0^t\textbf{B}_{L}\ 6 h\ dr ds $$

In [13]:
stiffness_linear = BL.T.dot(stiffness_m).dot(BL)*scale[0]*scale[1]*h

In [14]:
K = np.array([]).reshape(0,8);
for row in stiffness_linear:
    nodal_row = []
    for element in row:
        integration_over_volume= integrate(integrate(element,(x,-1,1)),(y,-1,1))
        nodal_row.append(integration_over_volume)
    K = np.concatenate((K,np.array([nodal_row])));
K =K.T
print(round_expr(K[0][0],4))

0.682*E*h


$$ {}^t_0\textbf{B}_{NL} = \begin{bmatrix}
h_{1,1} & 0 &|& ...\\
h_{1,2} & 0 &|& ...\\
0 & h_{1,1} &|& ...\\
0 & h_{1,2} &|& ...
\end{bmatrix} $$

In [15]:
BNL = np.array([]).reshape(0,4)
for hi in list_of_interp_fcts:
    BNL=np.concatenate((BNL,np.array([
        [hprime(hi,x,y,0),hprime(hi,x,y,1),0,0],
        [0,0,hprime(hi,x,y,0),hprime(hi,x,y,1)]
    ])))
BNL =BNL.T

In [16]:
exploded_piola = np.array([
    [piola_stress[0][0], piola_stress[0][1], 0,0],
    [piola_stress[0][1], piola_stress[1][1], 0,0],
    [0,0,piola_stress[0][0], piola_stress[0][1]],
    [0,0,piola_stress[0][0], piola_stress[0][1]]
    ])

$$ J = \frac{\partial^0x_1}{\partial r}\frac{\partial^0x_2}{\partial s} = 6 $$

$$ {}^t_0\textbf{K}_{NL} = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{NL}\textbf{S}_{exp}{}_0^t\textbf{B}_{NL}\ 6h \ dr ds $$

In [17]:
stiffness_non_linear = BNL.T.dot(exploded_piola).dot(BNL) *scale[0]*scale[1]*h

In [18]:
K_NL = np.array([]).reshape(0,8);
for row in stiffness_non_linear:
    nodal_row = []
    for element in row:
        integration_over_volume= integrate(integrate(element,(x,-1,1)),(y,-1,1))
        nodal_row.append(integration_over_volume)
    K_NL = np.concatenate((K_NL,np.array([nodal_row])));
K_NL=K_NL.T

$$ {}^t_0\textbf{K} = {}^t_0\textbf{K}_{L} + {}^t_0\textbf{K}_{NL} $$

In [19]:
K=K + K_NL
print(round_expr(K[0][0],4))

0.682*E*h + 52.2222*h


In [20]:
stress_piola_v = [piola_stress[0][0],piola_stress[1][1],piola_stress[0][1]]

$$ {}^t_0\textbf{F} = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{L}{}_0^t\textbf{S}_{vec} \ 6h \ dr ds $$

In [21]:
force_per_volume = BL.T.dot(stress_piola_v) *scale[0]*scale[1]*h

In [22]:
nodal_forces = np.empty(8,dtype=Symbol);
for counter, element in enumerate(force_per_volume):
    nodal_forces[counter] = integrate(integrate(element,(x,-1.0,1.0)),(y,-1.0,1.0))

In [23]:
print(nodal_forces[0])

250.0*h
